In [ ]:
# see https://github.com/pydantic/pydantic/discussions/9509

# this doesn't work for us because using __init__ functions that handle positional rather
# than keyword construction, as we do, interferes with Pydantic's type checking. Pending
# # asking the Pydantic folks for help, we kludge by naming the the field with the alias
#
# ...and when we can, we make users go to keyword rather than positional arguments.

from pydantic import BaseModel, ConfigDict, Field, ValidationError

from camdkit.compatibility import CompatibleBaseModel

# json_data = '{"lens_encoders" : 42}'
# json_data_alias = '{"lensEncoders" : 42}'
json_data = {"lens_encoders" : 42}
json_data_alias = {"lensEncoders" : 42}

class Model(BaseModel):

    model_config = ConfigDict(validate_assignment=True,
                              use_enum_values=True,
                              populate_by_name=True,
                              extra="forbid")

    lens_encoders: int = Field(alias="lensEncoders")

m0 = Model(lens_encoders=42)
print(f"m0.model_dump() -> {m0.model_dump()}")
assert(json_data == m0.model_dump())
print(f"m0.model_dump(by_alias=True) -> {m0.model_dump(by_alias=True)}")
assert(json_data_alias == m0.model_dump(by_alias=True))

try:
    m1 = Model.model_validate(json_data)
    print(f"validated {json_data} against model")
except ValidationError as e:
    print(f"failed to validate {json_data} against model: {e}")

try:
    m2 = Model.model_validate(json_data_alias)
    print(f"validated {json_data_alias} against model")
except ValidationError as e:
    print(f"failed to validate {json_data_alias} against model: {e}")

try:
    m0.lens_encoders = 65
    print("assigned to m0.lens_encoders OK")
except ValidationError as e:
    print(f"failed to assign to m0.lens_encoders: {e}")

# try:
#     m0.lensEncoders = 65
#     print("assigned to m0.lensEncoders OK")
# except ValidationError as e:
#     print(opt_param_fn"failed to assign to m0.lens_encoders: {e}")


In [ ]:
from typing import Annotated
from pydantic import BaseModel, Field
class XModel(BaseModel):
    model_config = ConfigDict(validate_assignment=True,
                              use_enum_values=True,
                              extra="forbid")
    x: Annotated[int, Field(serialization_alias="fred")]
    y: int
foo = XModel(x=1, y=2)
print(f"{foo=}")
bar = XModel(x=1, y=2)
print(f"{bar=}")



In [ ]:
from typing import Any
from pydantic import BaseModel
from camdkit.clip import Clip, Static

def get_it(self):
    print("in get_it")
    return "foo"

def set_it(self, value) -> None:
    print("in set_it")

def get_thang(self):
    print("in get_thang")
    return self.thing

def set_thang(self, value) -> None:
    print("in set_thang")
    self.thing = value


class Normal:
    def __init__(self):
        self.thing: str = "myself"

    it = property(get_it, set_it)


class Model(BaseModel):
    thing: str | None = "myself"

    it = property(get_it, set_it)


n = Normal()
try:
    print(f"{n.it=}")
except NameError:
    pass
Normal.thang = property(get_thang, set_thang)
n1 = Normal()
try:
    print(f"{n1.thang=}")
except NameError:
    pass
print("")

m = Model()
try:
    print(f"{m.it=}")
except NameError:
    pass
try:
    print(f"{m.thang=}")
except AttributeError as e:
    print(e)
Model.thang = property(get_thang, set_thang)
m1 = Model()
try:
    print(f"{m1.thang=}")
except NameError:
    pass
print("")

c0 = Clip()
try:
    print(f"{c0.thang=}")
except AttributeError as e:
    print(e)
print('adding thang property')
Clip.thang = property(get_it, set_it)
c1 = Clip()
try:
    try:
        print(f"{c1.thang=}")
    except AttributeError:
        print('No thing attribute for Clip, so get_thang should fail')
except NameError:
    pass
print("")


def add_property(cls, name: str, hierarchy: tuple[tuple[str, type], ...]):
    def get_thru_hierarchy(self):
        print("inside get_thru_hierarchy")
        obj = self
        field_names: list[str] = [field_name for field_name, model in hierarchy]
        field_names.append(name)
        for field_name in field_names:
            try:
                obj = getattr(obj, field_name)
            except AttributeError:
                return None
        return obj

    def set_thru_hierarchy(self, value: Any) -> None:
        obj = self
        if hierarchy:
            *field_names_and_models, field_name = hierarchy
            for model_field_name, model in field_names_and_models:
                if not hasattr(obj, model_field_name) or getattr(obj, model_field_name) is None:
                    setattr(obj, model_field_name, model())
                obj = getattr(obj, model_field_name)
        setattr(obj, name, value)

    # Clip.name = property(get_thru_hierarchy, set_thru_hierarchy)
    setattr(Clip, name, property(get_thru_hierarchy, set_thru_hierarchy))

print("adding duration to clip")
add_property(Clip, 'duration', (('static', Static),))

c2 = Clip()
print("made c2")
try:
    try:
        print(f"{c2.duration=}")
    except AttributeError as e:
        print(f'Attribute error for Clip: {e}')
except NameError as e1:
    print(f'Name error for Clip: {e}')
    pass


In [5]:
from camdkit.clip import Clip
from camdkit.numeric_types import StrictlyPositiveRational

c0 = Clip()
print(f"{c0.duration=}")
c0.duration = StrictlyPositiveRational(24,1)
print(f"{c0.duration=}")
c0.duration = None
print(f"{c0.duration=}")
print("all done")


AttributeError: 'Clip' object has no attribute 'duration'